# 03 BMRA

Run BMRA based on global responses derived from the pathway activity predictions.

In [1]:
import pandas as pd
import numpy as np
import os
import bmra

In [2]:
data_dir_1 = "/home/jing/input_bmra/00_outputs_2020_bc3c"
data_dir_2 = "/home/jing/input_bmra/02_outputs_2020_bc3c"
info_dir = "/home/jing/input_bmra/DATA"
out_dir = "/home/jing/input_bmra/03_outputs_2020_bc3c"


os.makedirs(out_dir, exist_ok = True)

## Load data

### Global responses

Assembled from global responses calculated from predicted pathway activities, and DPD global responses.

In [3]:
def assemble_R_global(pathway_rglobal_file, dpd_rglobal_file):
    """
    Assemble full R_global matrix
    """
    R_global_modules_df = pd.read_csv(pathway_rglobal_file, index_col = 0)

    perts = R_global_modules_df.columns.tolist()

    R_global_DPD_df = pd.read_csv(dpd_rglobal_file, index_col=0).T

    # filter by perts list
    R_global_DPD_df = R_global_DPD_df[R_global_DPD_df.index.isin(perts)]

    # sort by perts list
    R_global_DPD_df["sort_col"] = R_global_DPD_df.index.map({val: i for i, val in enumerate(perts)})
    R_global_DPD_df = R_global_DPD_df.sort_values("sort_col")
    R_global_DPD_df = R_global_DPD_df.drop("sort_col", axis = 1)

    # transpose
    R_global_DPD_df = R_global_DPD_df.T

    R_global_df = pd.concat([R_global_modules_df, R_global_DPD_df])

    return R_global_df

In [4]:
R_global_df = assemble_R_global(
    pathway_rglobal_file = os.path.join(data_dir_2, "R_global_annotated.csv"),
    dpd_rglobal_file = os.path.join(data_dir_1, "R_global_DPDonly_annotated.csv"),
)

modules = R_global_df.index.tolist()
exp_ids = R_global_df.columns.tolist()

R_global = R_global_df.values
n_modules = R_global.shape[0]

print(R_global.shape)
display(R_global_df)

(15, 263)


,CPC006_THP1_6H:C04,CPC006_THP1_6H:G06,CPC006_THP1_6H:K09,CPC006_THP1_6H:M21,EMU001_THP1_6H:B19,EMU001_THP1_3H:A22,EMU001_THP1_24H:B19,EMU001_THP1_6H:L19,EMU001_THP1_6H:L24,EMU001_THP1_24H:L24,...,AICHI001_THP1_4H:E19,AICHI001_THP1_4H:M15,AICHI002_THP1_24H:M09,AICHI001_THP1_24H:E22,AICHI001_THP1_24H:E23,AICHI001_THP1_4H:O14,AICHI001_THP1_24H:M18,AICHI001_THP1_4H:M13,AICHI001_THP1_4H:E24,AICHI001_THP1_4H:O17
CDK1/2,0.374603,-0.137492,-0.533110,0.065439,0.237383,0.284852,-0.167230,0.171545,0.144066,0.038893,...,-0.052059,-0.072298,-1.854854,0.104884,0.339705,-0.170311,-0.209217,-0.028539,-0.125735,-0.031983
CDK4/6,-0.118398,-0.027737,0.059306,0.024034,0.085174,-0.424450,0.019402,0.071208,0.104277,0.098970,...,-0.198745,-0.309989,-0.073639,-0.114822,0.236238,-0.138187,-0.153515,0.072525,-0.285714,-0.006866
IKK,-0.189119,-0.170653,-1.757087,0.187261,0.253872,-0.237323,0.236512,0.198977,0.067704,0.009282,...,0.432883,0.470171,0.671279,0.092816,-0.147237,0.546892,-0.108704,0.538753,0.334979,0.371028
TLR4,0.710553,-0.740479,0.321673,0.370013,-1.355629,-1.796879,-0.619664,-1.974616,-1.900657,-0.743977,...,-0.169788,-0.124031,0.598956,0.182503,-0.011150,-0.314755,0.278931,0.353600,-0.637882,-0.217086
mTOR,-1.974737,-1.961689,-0.279687,-1.273997,-1.980374,-0.774240,-1.623362,-1.526877,-1.061128,-0.756345,...,-0.454862,0.526461,0.711704,-0.368525,0.050514,-1.778228,-0.066710,0.445952,-0.427145,-0.276780
PAK,0.083215,-0.056729,0.259757,-0.195724,-0.347050,-0.085756,0.154775,0.184581,0.035799,0.045928,...,-0.001570,0.033059,-0.479381,-0.021239,-0.029220,-0.050557,0.028826,0.035146,0.076766,0.063774
ERK,-0.437640,-1.886647,-0.566214,-0.650670,-0.239301,-0.072744,0.249141,-0.090062,-0.058921,0.003748,...,-0.188757,-0.253064,-0.512933,0.051010,-0.027184,-0.282285,-0.039250,-0.837552,-0.122813,-0.140757
JAK/STAT,0.698184,0.424347,0.179488,-0.090389,-0.733237,-0.590702,0.146477,-1.713002,-0.165344,-0.329542,...,-1.678740,-1.849269,1.181472,-1.060296,-0.535668,-1.297214,-0.178899,-1.839350,-0.339628,-1.567789
PLK1,0.080809,-0.255565,0.376429,-0.152404,-0.144975,0.246421,-0.162250,0.098107,0.145986,-0.033686,...,0.100728,-0.227022,-0.726150,0.003315,-0.148821,0.140836,-0.062592,-0.095102,0.201962,0.228472
ITK,-0.306057,0.568791,-0.770360,0.079526,0.680750,0.587803,-0.151320,0.808088,0.704316,0.013359,...,0.487184,0.662892,-0.445331,0.060170,-0.053005,0.371950,-0.066572,0.635898,0.524521,0.505959


In [5]:
print(R_global_df.values.min())
print(R_global_df.values.max())

-3.8112734524695258
7.508320123223826


### Perturbation matrix

In [6]:
inhib_conc_df = pd.read_csv(os.path.join(data_dir_1, "inhib_conc_annotated.csv"), index_col = 0)
#LFC_PLCg = pd.read_csv(os.path.join(info_dir,"PLCg_Data_log_2020.csv"),index_col=0)
#inhib_conc = inhib_conc_df.drop(columns=LFC_PLCg.index).values
inhib_conc = inhib_conc_df.values

n_DPDs = R_global.shape[0] - inhib_conc.shape[0]

pert_modules = np.where(inhib_conc != 0, 1, 0)
pert_DPD = np.zeros((n_DPDs, len(exp_ids)))

pert = np.vstack([pert_modules, pert_DPD])

print(pert.shape)
print(pert)

(15, 263)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Networks

In [7]:
# empty prior
#G_prior = np.zeros((n_modules, n_modules))
G_prior_df = pd.read_excel("/home/jing/input_bmra/DATA/ALL_DATA_2020_Jing.xlsx", sheet_name = "prior_net_work", index_col = 0)

G_prior = G_prior_df.values
G_prior_df

,CDK1/2,CDK4/6,IKK,TLR4,mTOR,PAK,ERK,JAK/STAT,PLK1,ITK,DPD_resist,DPD_TB,DPD_load,DPD_damage,DPD_microa
CDK1/2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
CDK4/6,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
IKK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TLR4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
mTOR,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PAK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ERK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
JAK/STAT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
PLK1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ITK,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
#G_prior_r_df = pd.read_excel(os.path.join(info_dir, "ALL_DATA_2020.xlsx"), sheet_name = "prior_r", index_col = 0)

#G_prior_r = G_prior_r_df.values
#G_prior_r_df

In [9]:
G_not = np.eye(n_modules)
G_not[:, n_modules - n_DPDs :] = 1

G_not

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0.

## Run BMRA

In [10]:
n_gibbs = 200_000
n_window = 5_000

mode = "all"
n_iterations = 1

In [11]:
A_mean, A_sd, r_mean, r_sd = bmra.run_bmra(R_global,
            pert,
            G_prior,
            G_not,
            #r_prior = G_prior_r,
            n_gibbs = n_gibbs,
            n_window = n_window,
            mode = mode,
            n_iterations = n_iterations,
        )

A_mean_df = pd.DataFrame(A_mean, index = modules, columns = modules)
A_sd_df = pd.DataFrame(A_sd, index = modules, columns = modules)
r_mean_df = pd.DataFrame(r_mean, index = modules, columns = modules)
r_sd_df = pd.DataFrame(r_sd, index = modules, columns = modules)

display(A_mean_df)
display(A_sd_df)
display(r_mean_df)
display(r_sd_df)

A_mean_df.to_csv(os.path.join(out_dir, "A_mean.csv"))
A_sd_df.to_csv(os.path.join(out_dir, "A_sd.csv"))
r_mean_df.to_csv(os.path.join(out_dir, "r_mean.csv"))
r_sd_df.to_csv(os.path.join(out_dir, "r_sd.csv"))

,CDK1/2,CDK4/6,IKK,TLR4,mTOR,PAK,ERK,JAK/STAT,PLK1,ITK,DPD_resist,DPD_TB,DPD_load,DPD_damage,DPD_microa
CDK1/2,0.000000,0.260300,0.000000,1.000000,0.000343,1.000000,0.974101,0.000000,0.113141,0.000000,0.0,0.0,0.0,0.0,0.0
CDK4/6,1.000000,0.000000,1.000000,1.000000,0.999883,1.000000,1.000000,0.014696,0.001134,1.000000,0.0,0.0,0.0,0.0,0.0
IKK,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.001140,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0
TLR4,0.000000,0.000000,0.000000,0.000000,1.000000,0.741722,1.000000,1.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0
mTOR,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
PAK,1.000000,1.000000,0.001366,0.001988,1.000000,0.000000,1.000000,0.017030,1.000000,0.001367,0.0,0.0,0.0,0.0,0.0
ERK,0.015262,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000811,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0
JAK/STAT,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0
PLK1,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0
ITK,0.240118,0.148564,1.000000,1.000000,0.000151,0.947338,1.000000,1.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0


,CDK1/2,CDK4/6,IKK,TLR4,mTOR,PAK,ERK,JAK/STAT,PLK1,ITK,DPD_resist,DPD_TB,DPD_load,DPD_damage,DPD_microa
CDK1/2,0.000000e+00,4.387980e-01,0.000000e+00,6.661338e-16,1.852128e-02,6.661338e-16,1.588333e-01,0.000000e+00,3.167657e-01,0.000000e+00,0.0,0.0,0.0,0.0,0.0
CDK4/6,2.553513e-15,0.000000e+00,2.553513e-15,2.553513e-15,1.080892e-02,2.553513e-15,2.553513e-15,1.203346e-01,3.365800e-02,2.553513e-15,0.0,0.0,0.0,0.0,0.0
IKK,1.110223e-15,1.110223e-15,0.000000e+00,0.000000e+00,1.110223e-15,0.000000e+00,1.110223e-15,3.374657e-02,0.000000e+00,1.110223e-15,0.0,0.0,0.0,0.0,0.0
TLR4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.881784e-16,4.376874e-01,8.881784e-16,8.881784e-16,0.000000e+00,8.881784e-16,0.0,0.0,0.0,0.0,0.0
mTOR,2.664535e-15,0.000000e+00,0.000000e+00,2.664535e-15,0.000000e+00,2.664535e-15,0.000000e+00,2.664535e-15,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0
PAK,3.108624e-15,3.108624e-15,3.693797e-02,4.454651e-02,3.108624e-15,0.000000e+00,3.108624e-15,1.293814e-01,3.108624e-15,3.695192e-02,0.0,0.0,0.0,0.0,0.0
ERK,1.225911e-01,2.442491e-15,2.442491e-15,2.442491e-15,2.442491e-15,0.000000e+00,0.000000e+00,2.846475e-02,2.442491e-15,2.442491e-15,0.0,0.0,0.0,0.0,0.0
JAK/STAT,2.220446e-16,0.000000e+00,0.000000e+00,2.220446e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.220446e-16,0.0,0.0,0.0,0.0,0.0
PLK1,4.440892e-16,0.000000e+00,4.440892e-16,0.000000e+00,4.440892e-16,4.440892e-16,4.440892e-16,0.000000e+00,0.000000e+00,4.440892e-16,0.0,0.0,0.0,0.0,0.0
ITK,4.271549e-01,3.556578e-01,2.220446e-15,2.220446e-15,1.229536e-02,2.233572e-01,2.220446e-15,2.220446e-15,2.220446e-15,0.000000e+00,0.0,0.0,0.0,0.0,0.0


,CDK1/2,CDK4/6,IKK,TLR4,mTOR,PAK,ERK,JAK/STAT,PLK1,ITK,DPD_resist,DPD_TB,DPD_load,DPD_damage,DPD_microa
CDK1/2,0.000000,-0.006180,0.000000,-0.089441,-0.000007,0.113034,0.099631,0.000000,0.007766,0.000000,0.0,0.0,0.0,0.0,0.0
CDK4/6,0.143812,0.000000,0.338529,0.070591,-0.034364,0.159793,0.194604,0.000454,-0.000025,-0.143824,0.0,0.0,0.0,0.0,0.0
IKK,-0.222665,0.359726,0.000000,0.000000,-0.065159,0.000000,0.216279,-0.000044,0.000000,0.308449,0.0,0.0,0.0,0.0,0.0
TLR4,0.000000,0.000000,0.000000,0.000000,0.284653,-0.124210,-0.275527,0.276155,0.000000,-0.198946,0.0,0.0,0.0,0.0,0.0
mTOR,-0.293406,0.000000,0.000000,0.305921,0.000000,0.667978,0.000000,0.195370,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
PAK,0.233641,0.111028,-0.000046,0.000009,0.037056,0.000000,-0.173905,-0.000386,0.157598,-0.000030,0.0,0.0,0.0,0.0,0.0
ERK,0.001238,0.210032,0.317996,-0.104185,0.103864,0.000000,0.000000,0.000048,0.408240,-0.244670,0.0,0.0,0.0,0.0,0.0
JAK/STAT,-0.238836,0.000000,0.000000,0.220427,0.000000,0.000000,0.000000,0.000000,0.000000,-0.315616,0.0,0.0,0.0,0.0,0.0
PLK1,0.119018,0.000000,-0.148267,0.000000,-0.089469,0.314287,0.665025,0.000000,0.000000,0.231009,0.0,0.0,0.0,0.0,0.0
ITK,0.022813,-0.018539,0.361711,-0.127635,-0.000006,-0.119685,-0.276960,-0.157788,0.163154,0.000000,0.0,0.0,0.0,0.0,0.0


,CDK1/2,CDK4/6,IKK,TLR4,mTOR,PAK,ERK,JAK/STAT,PLK1,ITK,DPD_resist,DPD_TB,DPD_load,DPD_damage,DPD_microa
CDK1/2,0.000000,0.018249,0.000000,0.007486,0.000384,0.017042,0.022532,0.000000,0.023095,0.000000,0.0,0.0,0.0,0.0,0.0
CDK4/6,0.023951,0.000000,0.027544,0.015418,0.016055,0.025690,0.018033,0.004088,0.000819,0.025941,0.0,0.0,0.0,0.0,0.0
IKK,0.018137,0.023365,0.000000,0.000000,0.011558,0.000000,0.014294,0.001302,0.000000,0.019911,0.0,0.0,0.0,0.0,0.0
TLR4,0.000000,0.000000,0.000000,0.000000,0.025965,0.080330,0.022328,0.028600,0.000000,0.035053,0.0,0.0,0.0,0.0,0.0
mTOR,0.021468,0.000000,0.000000,0.014268,0.000000,0.022289,0.000000,0.015587,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
PAK,0.015982,0.019904,0.001281,0.000356,0.009938,0.000000,0.015862,0.003157,0.015238,0.000914,0.0,0.0,0.0,0.0,0.0
ERK,0.010316,0.037191,0.040605,0.019005,0.019591,0.000000,0.000000,0.001709,0.022288,0.036734,0.0,0.0,0.0,0.0,0.0
JAK/STAT,0.016041,0.000000,0.000000,0.010841,0.000000,0.000000,0.000000,0.000000,0.000000,0.018225,0.0,0.0,0.0,0.0,0.0
PLK1,0.021650,0.000000,0.023857,0.000000,0.012464,0.022577,0.015116,0.000000,0.000000,0.021563,0.0,0.0,0.0,0.0,0.0
ITK,0.042423,0.045667,0.030628,0.014564,0.000470,0.038330,0.025919,0.017324,0.024284,0.000000,0.0,0.0,0.0,0.0,0.0


In [12]:
np.fill_diagonal(r_mean_df.values,-1)
rm_minus_inv = pd.DataFrame(np.linalg.pinv(r_mean_df),index=r_mean_df.index,columns=r_mean_df.columns)*(-1)
rm_minus_inv.to_csv(os.path.join(out_dir, 'r_minv.csv'))
display(rm_minus_inv)

,CDK1/2,CDK4/6,IKK,TLR4,mTOR,PAK,ERK,JAK/STAT,PLK1,ITK,DPD_resist,DPD_TB,DPD_load,DPD_damage,DPD_microa
CDK1/2,1.070377,0.098171,0.100630,-0.138134,-0.030489,0.172891,0.238128,-0.049432,0.138265,0.033675,1.317576e-16,1.704880e-16,3.338688e-16,-2.510598e-17,9.466358e-17
CDK4/6,0.172332,1.341999,0.545800,0.034785,-0.027710,0.301949,0.529164,0.022211,0.246798,-0.111058,3.236202e-16,-9.743404e-17,-5.372116e-16,-3.146626e-16,-1.050246e-15
IKK,-0.115346,0.648635,1.476664,-0.103897,-0.114574,0.076622,0.546149,-0.106072,0.291224,0.349980,-8.119300e-18,-6.350974e-16,-2.300832e-16,3.313475e-16,-4.342529e-16
TLR4,-0.246354,-0.273611,-0.404709,1.357987,0.385261,-0.036704,-0.636462,0.519983,-0.339718,-0.442515,-3.241428e-16,1.513121e-16,-3.323238e-16,-4.919720e-16,-7.899423e-17
mTOR,-0.257144,-0.054920,-0.190872,0.525376,1.161262,0.679497,-0.320712,0.406306,-0.061652,-0.219527,-5.466398e-16,-4.744804e-16,-1.253299e-16,2.197666e-17,-1.827421e-16
PAK,0.293164,0.123785,0.023221,-0.018794,0.011831,1.112692,-0.026200,-0.010009,0.173921,0.042812,4.532378e-17,4.082670e-17,-1.994769e-16,1.788809e-16,-8.493427e-17
ERK,0.135166,0.649465,0.626907,-0.128036,0.032357,0.416245,1.790890,-0.008202,0.776404,-0.130813,-1.308182e-17,-4.309692e-16,2.124696e-17,3.092141e-16,-4.503158e-16
JAK/STAT,-0.325290,-0.128466,-0.271533,0.419534,0.135917,-0.009221,-0.197760,1.225439,-0.170616,-0.526538,-3.782750e-16,-3.239907e-16,1.039663e-16,-6.268207e-16,1.193536e-17
PLK1,0.360758,0.423964,0.350179,-0.202923,-0.097297,0.545544,1.159246,-0.107581,1.595682,0.206329,-3.815210e-16,-3.781127e-16,1.075563e-16,1.087014e-16,-3.654697e-16
ITK,0.048610,0.141652,0.501529,-0.276301,-0.138502,-0.127251,0.001878,-0.313720,0.198692,1.333751,1.785239e-16,-4.668509e-16,-1.782604e-16,-2.491266e-16,6.045449e-17


In [13]:
con_mat = pd.DataFrame(columns=['From','To','Strength'])
for con_to in r_mean_df.index:
    for con_from in r_mean_df.columns:
        if ((r_mean_df.loc[con_to,con_from] != 0) & (con_from != con_to)):
            row_df = pd.DataFrame([[con_from,con_to,r_mean_df.loc[con_to,con_from]]],columns=['From','To','Strength'])
            if con_mat.empty:
                con_mat = row_df.copy()
            else:
                con_mat = pd.concat([con_mat,row_df],axis=0,ignore_index=True)
            #con_mat = con_mat.append({'From':con_from,'To':con_to,'Strength':r_mean_df.loc[con_to,con_from]},ignore_index=True)
con_mat.to_csv(os.path.join(out_dir, 'r_net.txt'),sep='\t',index=False)
display(con_mat)

,From,To,Strength
0,CDK4/6,CDK1/2,-0.006180
1,TLR4,CDK1/2,-0.089441
2,mTOR,CDK1/2,-0.000007
3,PAK,CDK1/2,0.113034
4,ERK,CDK1/2,0.099631
...,...,...,...
91,IKK,DPD_microa,-0.686179
92,ERK,DPD_microa,-0.545829
93,JAK/STAT,DPD_microa,-0.473638
94,PLK1,DPD_microa,-0.397271


In [14]:
# now the same but with filtered matrices
# threshold value
Athr = 0.5

rm_filt = r_mean_df[np.abs(A_mean_df) > Athr].fillna(0).copy()
np.fill_diagonal(rm_filt.values,-1)
rm_filt.to_csv(os.path.join(out_dir, 'r_mean_filt.csv'))
display(rm_filt)

rm_filt_minus_inv = pd.DataFrame(np.linalg.pinv(rm_filt),index=rm_filt.index,columns=rm_filt.columns)*(-1)
rm_filt_minus_inv.to_csv(os.path.join(out_dir, 'r_minv_filt.csv'))
display(rm_minus_inv)

con_mat = pd.DataFrame(columns=['From','To','Strength'])
for con_to in rm_filt.index:
    for con_from in rm_filt.columns:
        if ((rm_filt.loc[con_to,con_from] != 0) & (con_from != con_to)):
            row_df = pd.DataFrame([[con_from,con_to,rm_filt.loc[con_to,con_from]]],columns=['From','To','Strength'])
            if con_mat.empty:
                con_mat = row_df.copy()
            else:
                con_mat = pd.concat([con_mat,row_df],axis=0,ignore_index=True)
            #con_mat = con_mat.append({'From':con_from,'To':con_to,'Strength':r_mean_df.loc[con_to,con_from]},ignore_index=True)
con_mat.to_csv(os.path.join(out_dir, 'r_net_filt.txt'),sep='\t',index=False)
#display(con_mat)


,CDK1/2,CDK4/6,IKK,TLR4,mTOR,PAK,ERK,JAK/STAT,PLK1,ITK,DPD_resist,DPD_TB,DPD_load,DPD_damage,DPD_microa
CDK1/2,-1.000000,0.000000,0.000000,-0.089441,0.000000,0.113034,0.099631,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
CDK4/6,0.143812,-1.000000,0.338529,0.070591,-0.034364,0.159793,0.194604,0.000000,0.000000,-0.143824,0.0,0.0,0.0,0.0,0.0
IKK,-0.222665,0.359726,-1.000000,0.000000,-0.065159,0.000000,0.216279,0.000000,0.000000,0.308449,0.0,0.0,0.0,0.0,0.0
TLR4,0.000000,0.000000,0.000000,-1.000000,0.284653,-0.124210,-0.275527,0.276155,0.000000,-0.198946,0.0,0.0,0.0,0.0,0.0
mTOR,-0.293406,0.000000,0.000000,0.305921,-1.000000,0.667978,0.000000,0.195370,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
PAK,0.233641,0.111028,0.000000,0.000000,0.037056,-1.000000,-0.173905,0.000000,0.157598,0.000000,0.0,0.0,0.0,0.0,0.0
ERK,0.000000,0.210032,0.317996,-0.104185,0.103864,0.000000,-1.000000,0.000000,0.408240,-0.244670,0.0,0.0,0.0,0.0,0.0
JAK/STAT,-0.238836,0.000000,0.000000,0.220427,0.000000,0.000000,0.000000,-1.000000,0.000000,-0.315616,0.0,0.0,0.0,0.0,0.0
PLK1,0.119018,0.000000,-0.148267,0.000000,-0.089469,0.314287,0.665025,0.000000,-1.000000,0.231009,0.0,0.0,0.0,0.0,0.0
ITK,0.000000,0.000000,0.361711,-0.127635,0.000000,-0.119685,-0.276960,-0.157788,0.163154,-1.000000,0.0,0.0,0.0,0.0,0.0


,CDK1/2,CDK4/6,IKK,TLR4,mTOR,PAK,ERK,JAK/STAT,PLK1,ITK,DPD_resist,DPD_TB,DPD_load,DPD_damage,DPD_microa
CDK1/2,1.070377,0.098171,0.100630,-0.138134,-0.030489,0.172891,0.238128,-0.049432,0.138265,0.033675,1.317576e-16,1.704880e-16,3.338688e-16,-2.510598e-17,9.466358e-17
CDK4/6,0.172332,1.341999,0.545800,0.034785,-0.027710,0.301949,0.529164,0.022211,0.246798,-0.111058,3.236202e-16,-9.743404e-17,-5.372116e-16,-3.146626e-16,-1.050246e-15
IKK,-0.115346,0.648635,1.476664,-0.103897,-0.114574,0.076622,0.546149,-0.106072,0.291224,0.349980,-8.119300e-18,-6.350974e-16,-2.300832e-16,3.313475e-16,-4.342529e-16
TLR4,-0.246354,-0.273611,-0.404709,1.357987,0.385261,-0.036704,-0.636462,0.519983,-0.339718,-0.442515,-3.241428e-16,1.513121e-16,-3.323238e-16,-4.919720e-16,-7.899423e-17
mTOR,-0.257144,-0.054920,-0.190872,0.525376,1.161262,0.679497,-0.320712,0.406306,-0.061652,-0.219527,-5.466398e-16,-4.744804e-16,-1.253299e-16,2.197666e-17,-1.827421e-16
PAK,0.293164,0.123785,0.023221,-0.018794,0.011831,1.112692,-0.026200,-0.010009,0.173921,0.042812,4.532378e-17,4.082670e-17,-1.994769e-16,1.788809e-16,-8.493427e-17
ERK,0.135166,0.649465,0.626907,-0.128036,0.032357,0.416245,1.790890,-0.008202,0.776404,-0.130813,-1.308182e-17,-4.309692e-16,2.124696e-17,3.092141e-16,-4.503158e-16
JAK/STAT,-0.325290,-0.128466,-0.271533,0.419534,0.135917,-0.009221,-0.197760,1.225439,-0.170616,-0.526538,-3.782750e-16,-3.239907e-16,1.039663e-16,-6.268207e-16,1.193536e-17
PLK1,0.360758,0.423964,0.350179,-0.202923,-0.097297,0.545544,1.159246,-0.107581,1.595682,0.206329,-3.815210e-16,-3.781127e-16,1.075563e-16,1.087014e-16,-3.654697e-16
ITK,0.048610,0.141652,0.501529,-0.276301,-0.138502,-0.127251,0.001878,-0.313720,0.198692,1.333751,1.785239e-16,-4.668509e-16,-1.782604e-16,-2.491266e-16,6.045449e-17


In [15]:
# number of zero elements in the matrix
display(len(rm_filt.index)*len(rm_filt.columns) - rm_filt.astype(bool).sum().sum())

135

In [16]:
# number of non-zero elements in the matrix
display(rm_filt.astype(bool).sum().sum() - len(modules))

75